# Experiment 023: NFP-based Chebyshev Packing

Based on web search results, top teams achieving scores below 69 use:
1. Square-packing with Chebyshev distance
2. Smart scan-line linear packing
3. No-Fit Polygon (NFP) based placement

This experiment implements a greedy constructive heuristic using Chebyshev distance to minimize bounding box.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.ops import unary_union
import json
from tqdm import tqdm

# Tree template
TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree_polygon(x, y, angle):
    """Create a tree polygon at position (x, y) with given rotation angle."""
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_tree_bounds(angle):
    """Get bounding box dimensions for a tree at given angle."""
    tree = create_tree_polygon(0, 0, angle)
    minx, miny, maxx, maxy = tree.bounds
    return maxx - minx, maxy - miny

def check_overlap(tree1, tree2):
    """Check if two trees overlap (touching is allowed)."""
    return tree1.overlaps(tree2) or tree1.contains(tree2) or tree2.contains(tree1)

def get_bounding_box_side(trees):
    """Get the side length of the bounding box containing all trees."""
    all_x = []
    all_y = []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    if not all_x:
        return 0
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score_contribution(side, n):
    """Calculate score contribution for a single N."""
    return (side ** 2) / n

print("Functions defined")

Functions defined


In [2]:
# Load current best solution for comparison
def parse_s_value(val):
    if isinstance(val, str):
        if val.startswith('s'):
            return float(val[1:])
        return float(val)
    return float(val)

best_df = pd.read_csv('/home/submission/submission.csv')
best_df['x'] = best_df['x'].apply(parse_s_value)
best_df['y'] = best_df['y'].apply(parse_s_value)
best_df['deg'] = best_df['deg'].apply(parse_s_value)
best_df['n'] = best_df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Loaded best solution with {len(best_df)} rows")

# Calculate best scores for each N
best_scores = {}
best_sides = {}
for n in range(1, 201):
    group = best_df[best_df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    best_sides[n] = side
    best_scores[n] = calculate_score_contribution(side, n)

print(f"Best total score: {sum(best_scores.values()):.6f}")

Loaded best solution with 20100 rows


Best total score: 70.627608


In [3]:
# Greedy Chebyshev Packing
# The idea: place trees one by one, choosing position that minimizes Chebyshev distance from center
# Chebyshev distance = max(|x|, |y|) - this minimizes the square bounding box

def greedy_chebyshev_pack(n, angles, spacing=0.9):
    """
    Greedy packing using Chebyshev distance to minimize bounding box.
    
    Args:
        n: Number of trees
        angles: List of angles for each tree
        spacing: Minimum spacing between trees
    
    Returns:
        List of (x, y, angle) tuples, or None if packing fails
    """
    if n == 0:
        return []
    
    # Place first tree at origin
    placements = [(0, 0, angles[0])]
    placed_trees = [create_tree_polygon(0, 0, angles[0])]
    
    for i in range(1, n):
        angle = angles[i]
        best_pos = None
        best_chebyshev = float('inf')
        
        # Try positions on a grid around existing trees
        # Use Chebyshev distance to prioritize positions closer to center
        for dx in np.arange(-5, 5.1, spacing * 0.5):
            for dy in np.arange(-5, 5.1, spacing * 0.5):
                # Chebyshev distance from origin
                cheb_dist = max(abs(dx), abs(dy))
                
                if cheb_dist >= best_chebyshev:
                    continue
                
                # Create candidate tree
                candidate = create_tree_polygon(dx, dy, angle)
                
                # Check for overlaps with existing trees
                has_overlap = False
                for placed in placed_trees:
                    if check_overlap(candidate, placed):
                        has_overlap = True
                        break
                
                if not has_overlap:
                    best_pos = (dx, dy)
                    best_chebyshev = cheb_dist
        
        if best_pos is None:
            # Try larger search area
            for dx in np.arange(-10, 10.1, spacing * 0.3):
                for dy in np.arange(-10, 10.1, spacing * 0.3):
                    cheb_dist = max(abs(dx), abs(dy))
                    
                    if cheb_dist >= best_chebyshev:
                        continue
                    
                    candidate = create_tree_polygon(dx, dy, angle)
                    
                    has_overlap = False
                    for placed in placed_trees:
                        if check_overlap(candidate, placed):
                            has_overlap = True
                            break
                    
                    if not has_overlap:
                        best_pos = (dx, dy)
                        best_chebyshev = cheb_dist
        
        if best_pos is None:
            return None  # Failed to place tree
        
        placements.append((best_pos[0], best_pos[1], angle))
        placed_trees.append(create_tree_polygon(best_pos[0], best_pos[1], angle))
    
    return placements

print("Greedy Chebyshev packing function defined")

Greedy Chebyshev packing function defined


In [4]:
# Test on small N values first
test_n_values = [5, 10, 15, 20]
results = {}

for n in test_n_values:
    print(f"\n=== Testing N={n} ===")
    best_n_score = best_scores[n]
    best_n_side = best_sides[n]
    print(f"Baseline: side={best_n_side:.6f}, score={best_n_score:.6f}")
    
    # Try different angle configurations
    best_found_side = float('inf')
    best_found_config = None
    
    # Strategy 1: Use baseline angles
    baseline_group = best_df[best_df['n'] == n]
    baseline_angles = baseline_group['deg'].tolist()
    
    placements = greedy_chebyshev_pack(n, baseline_angles, spacing=0.8)
    if placements:
        trees = [create_tree_polygon(x, y, a) for x, y, a in placements]
        side = get_bounding_box_side(trees)
        print(f"  Baseline angles: side={side:.6f}")
        if side < best_found_side:
            best_found_side = side
            best_found_config = placements
    
    # Strategy 2: Alternating 0/180 angles
    alt_angles = [0 if i % 2 == 0 else 180 for i in range(n)]
    placements = greedy_chebyshev_pack(n, alt_angles, spacing=0.8)
    if placements:
        trees = [create_tree_polygon(x, y, a) for x, y, a in placements]
        side = get_bounding_box_side(trees)
        print(f"  Alternating 0/180: side={side:.6f}")
        if side < best_found_side:
            best_found_side = side
            best_found_config = placements
    
    # Strategy 3: Optimal tessellation angles (~68/248)
    tess_angles = [68 if i % 2 == 0 else 248 for i in range(n)]
    placements = greedy_chebyshev_pack(n, tess_angles, spacing=0.8)
    if placements:
        trees = [create_tree_polygon(x, y, a) for x, y, a in placements]
        side = get_bounding_box_side(trees)
        print(f"  Tessellation 68/248: side={side:.6f}")
        if side < best_found_side:
            best_found_side = side
            best_found_config = placements
    
    best_found_score = calculate_score_contribution(best_found_side, n)
    improvement = best_n_score - best_found_score
    
    results[n] = {
        'baseline_side': best_n_side,
        'baseline_score': best_n_score,
        'found_side': best_found_side,
        'found_score': best_found_score,
        'improvement': improvement
    }
    
    print(f"  Best found: side={best_found_side:.6f}, score={best_found_score:.6f}")
    print(f"  Improvement: {improvement:.6f}")


=== Testing N=5 ===
Baseline: side=1.443692, score=0.416850
  Baseline angles: side=2.258690
  Alternating 0/180: side=2.200000
  Tessellation 68/248: side=3.083494
  Best found: side=2.200000, score=0.968000
  Improvement: -0.551150

=== Testing N=10 ===
Baseline: side=1.940696, score=0.376630
  Baseline angles: side=3.486535
  Alternating 0/180: side=3.600000
  Tessellation 68/248: side=3.483494
  Best found: side=3.483494, score=1.213473
  Improvement: -0.836843

=== Testing N=15 ===
Baseline: side=2.377955, score=0.376978


  Baseline angles: side=3.627982
  Alternating 0/180: side=4.000000


  Tessellation 68/248: side=3.883494
  Best found: side=3.627982, score=0.877484
  Improvement: -0.500506

=== Testing N=20 ===
Baseline: side=2.742469, score=0.376057
  Baseline angles: side=4.695445


  Alternating 0/180: side=4.400000


  Tessellation 68/248: side=5.083494
  Best found: side=4.400000, score=0.968000
  Improvement: -0.591943


In [5]:
# Print summary
print("\n=== SUMMARY ===")
total_improvement = 0
for n, r in results.items():
    imp_str = f"+{r['improvement']:.6f}" if r['improvement'] > 0 else f"{r['improvement']:.6f}"
    print(f"N={n}: baseline={r['baseline_score']:.6f}, found={r['found_score']:.6f}, improvement={imp_str}")
    if r['improvement'] > 0:
        total_improvement += r['improvement']

print(f"\nTotal improvement on test N values: {total_improvement:.6f}")

if total_improvement <= 0:
    print("\nGreedy Chebyshev packing did NOT improve on baseline.")
    print("The baseline solution has a fundamentally better structure.")
else:
    print(f"\nGreedy Chebyshev packing found {total_improvement:.6f} improvement!")


=== SUMMARY ===
N=5: baseline=0.416850, found=0.968000, improvement=-0.551150
N=10: baseline=0.376630, found=1.213473, improvement=-0.836843
N=15: baseline=0.376978, found=0.877484, improvement=-0.500506
N=20: baseline=0.376057, found=0.968000, improvement=-0.591943

Total improvement on test N values: 0.000000

Greedy Chebyshev packing did NOT improve on baseline.
The baseline solution has a fundamentally better structure.


In [6]:
# Save metrics
import json

metrics = {
    'cv_score': sum(best_scores.values()),  # No improvement, use baseline
    'test_results': {str(n): r for n, r in results.items()},
    'total_improvement': total_improvement,
    'approach': 'greedy_chebyshev_packing'
}

with open('/home/code/experiments/023_nfp_chebyshev/metrics.json', 'w') as f:
    json.dump(metrics, f)

print(f"Metrics saved: {metrics}")

Metrics saved: {'cv_score': 70.62760781485349, 'test_results': {'5': {'baseline_side': 1.443692472141104, 'baseline_score': 0.4168495908233784, 'found_side': 2.199999999999996, 'found_score': 0.9679999999999966, 'improvement': -0.5511504091766182}, '10': {'baseline_side': 1.940695856224301, 'baseline_score': 0.3766300406366173, 'found_side': 3.483494167306862, 'found_score': 1.213473161366093, 'improvement': -0.8368431207294756}, '15': {'baseline_side': 2.377955118980222, 'baseline_score': 0.3769780365256162, 'found_side': 3.627982261417264, 'found_score': 0.8774836859438884, 'improvement': -0.5005056494182722}, '20': {'baseline_side': 2.7424691778731254, 'baseline_score': 0.3760568595792048, 'found_side': 4.400000000000002, 'found_score': 0.9680000000000009, 'improvement': -0.591943140420796}}, 'total_improvement': 0, 'approach': 'greedy_chebyshev_packing'}


In [7]:
# Copy best submission (no improvement found)
import shutil
shutil.copy('/home/submission/submission.csv', '/home/code/experiments/023_nfp_chebyshev/submission.csv')
print("Submission saved (baseline, no improvement found)")

Submission saved (baseline, no improvement found)
